# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [2]:
# Load the saved features from the .npz file
data = np.load('eo_freq.npz') 

# Access the concatenated features from the .npz file
X_freq = data['X_frequency']

# Close the loaded file
data.close()

#print the merged array
X_freq

array([[[ 2.12420499e-13,  2.42850861e-21,  2.17333937e-25, ...,
          8.23476874e-01,  9.85782795e-01, -1.01484112e-05],
        [ 2.88868261e-13,  4.46869239e-21,  4.57016445e-25, ...,
          6.66880173e-01,  7.90972473e-01, -6.81969343e-06],
        [ 1.86182779e-13,  2.82449932e-22,  3.40782554e-25, ...,
          1.75973470e-01,  4.14786760e-01,  2.92669772e-06],
        ...,
        [ 2.11538773e-13,  2.35658533e-22,  4.52489763e-25, ...,
          9.93022355e-02,  6.79438378e-01, -8.20565579e-07],
        [ 8.96401499e-14,  4.59474596e-21,  5.52098893e-26, ...,
          1.39069033e+00,  1.65071552e+00,  5.38626585e-06],
        [ 2.40712249e-13,  2.63701876e-21,  3.76594332e-25, ...,
          2.49404487e-01,  1.09178829e+00,  3.14978639e-06]],

       [[ 1.76626563e-13,  2.88638031e-21,  1.57054965e-25, ...,
          7.45290278e-01,  2.43906552e+00,  3.86663628e-06],
        [ 1.98227447e-13,  1.07978167e-21,  1.39520094e-25, ...,
          4.87379832e+00,  8.49455674e

In [3]:
data = np.load('eyes_open_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('eyes_open_groups.npz')

group = data['group']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [5]:
Y=Y
groups_array = np.hstack(group)

In [6]:
# Check the shape of the concatenated arrays
print(X_freq.shape, Y.shape, groups_array.shape)

(18145, 20, 20) (18145,) (18145,)


In [7]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    # model.add(Dropout(dropout_rate))
    
    # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256)))
    # model.add(Dropout(dropout_rate))

    
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [10]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
import os
import csv

metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_freq, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_freq[train_index], X_freq[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

    # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.h5'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=200, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks=[tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1
Epoch 1/200
64/64 [==============================] - 9s 47ms/step - loss: 0.4627 - accuracy: 0.7726 - val_loss: 0.5199 - val_accuracy: 0.8391
Epoch 2/200
64/64 [==============================] - 1s 21ms/step - loss: 0.3563 - accuracy: 0.8329 - val_loss: 0.4241 - val_accuracy: 0.8342
Epoch 3/200
64/64 [==============================] - 1s 23ms/step - loss: 0.3184 - accuracy: 0.8564 - val_loss: 0.3102 - val_accuracy: 0.8744
Epoch 4/200
64/64 [==============================] - 1s 23ms/step - loss: 0.2924 - accuracy: 0.8659 - val_loss: 0.2870 - val_accuracy: 0.8843
Epoch 5/200
64/64 [==============================] - 1s 23ms/step - loss: 0.2796 - accuracy: 0.8786 - val_loss: 0.2579 - val_accuracy: 0.8920
Epoch 6/200
64/64 [==============================] - 1s 23ms/step - loss: 0.2580 - accuracy: 0.8841 - val_loss: 0.2522 - val_accuracy: 0.8970
Epoch 7/200
64/64 [==============================] - 1s 21ms/step - loss: 0.2420 - accuracy: 0.8942 - val_loss: 0.2775 - val_accuracy: 0.8837

In [11]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [12]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9526,0.9536,0.9556,0.9546,0.9525,0.9493,0.9556,0.9050,0.9050
1,0.9477,0.9551,0.9440,0.9495,0.9478,0.9516,0.9440,0.8952,0.8952
2,0.9455,0.9380,0.9588,0.9483,0.9448,0.9309,0.9588,0.8906,0.8908
3,0.9510,0.9487,0.9578,0.9532,0.9507,0.9435,0.9578,0.9017,0.9017
4,0.9592,0.9719,0.9493,0.9605,0.9597,0.9700,0.9493,0.9184,0.9187
5,0.9509,0.9506,0.9556,0.9531,0.9507,0.9459,0.9556,0.9017,0.9017
6,0.9443,0.9378,0.9567,0.9471,0.9438,0.9309,0.9567,0.8883,0.8885
7,0.9515,0.9507,0.9567,0.9537,0.9512,0.9458,0.9567,0.9028,0.9028
8,0.9559,0.9446,0.9725,0.9584,0.9551,0.9377,0.9725,0.9115,0.9119
9,0.9570,0.9550,0.9630,0.9590,0.9567,0.9504,0.9630,0.9138,0.9138


In [13]:
metrics_df.round(4).to_csv('TIME_EO_CNN_LSTM.csv', index = False)

In [14]:
# import csv
# for i in range (1, 11):
#     df = pd.read_csv(f'epoch_{i}.csv')
#     [col] = df.columns
#     split_data = df[col].str.split(';')
#     csv_file = (f'epoch_{i}.csv')
#     # Prepare CSV file and write headers
#     with open(csv_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
#     for j in range (0, 200):
#         with open(csv_file, mode='a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
result = pd.read_csv("TIME_EO_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9526,0.9536,0.9556,0.9546,0.9525,0.9493,0.9556,0.9050,0.9050
1,0.9477,0.9551,0.9440,0.9495,0.9478,0.9516,0.9440,0.8952,0.8952
2,0.9455,0.9380,0.9588,0.9483,0.9448,0.9309,0.9588,0.8906,0.8908
3,0.9510,0.9487,0.9578,0.9532,0.9507,0.9435,0.9578,0.9017,0.9017
4,0.9592,0.9719,0.9493,0.9605,0.9597,0.9700,0.9493,0.9184,0.9187
5,0.9509,0.9506,0.9556,0.9531,0.9507,0.9459,0.9556,0.9017,0.9017
6,0.9443,0.9378,0.9567,0.9471,0.9438,0.9309,0.9567,0.8883,0.8885
7,0.9515,0.9507,0.9567,0.9537,0.9512,0.9458,0.9567,0.9028,0.9028
8,0.9559,0.9446,0.9725,0.9584,0.9551,0.9377,0.9725,0.9115,0.9119
9,0.9570,0.9550,0.9630,0.9590,0.9567,0.9504,0.9630,0.9138,0.9138


In [17]:
(result.mean()*100).round(2)

Accuracy       95.16
Precision      95.06
Recall         95.70
F1             95.37
Sensitivity    95.13
Specificity    94.56
ROC_AUC        95.70
Kappa          90.29
MCC            90.30
dtype: float64